# Persistence and Consistency

## Crash Consistency

Since updates to our file system require multiple updates to the file system's data structures, a crash in the middle of the process could lead to the system ending inconsistent state.

## Example

To write a new block to a file.

We need to update:
- the data block BIT MAP
- the DATA BLOCK
- the INODE for that file

### Crash Cases

1) The DATA BLOCK is written, but nothing else.
    - The user's data has been written, but neither the BITMAP nor INODE have. 
    - This is as if the data hasn't been written as all since the inode doesn't
    refer to it. From the user's perspective, they lost data.
    - From the file system's perspective, everything is consistent. The inode
    and bitmap agree with each other.
2) the BITMAP is written, but nothing else.
    - The bitmap says the block is claimed, but NO file refer to it, so it will never be freed. 
    - This is a space leak. The file system is permanently 4KB smaller.
    - The indoes and bitmap disagrees. The file system is in an INCONSISTENT state.
3) the INODE is written, but nothing else.
    - The inodes says that the data block is part of the file, but the data block
    contains garbage. The file will likely get corrupted. 
    - Also, the bitmap hasn't been written to. This block is still listed as free and could be added to some other file. In this case, we could have two files referring to the
    same data block. Then it's possible for the second file to eventually get
    corrupted as well if the first ever writes to that shared data block.
    The user likely loses data AND the system is in an INCONSISTENT state.
4) The DATA BLOCK and BITMAP are written, but not the INODE.
    - The system will be INCONSISTENT since the metadata disagrees. 
    - The user loses data and the system permanently loses space since no files refer to this
    data block for it to be freed.
5) The DATA BLOCK and INODE are written, but not the BITMAP.
    - The system will be INCONSISTENT since the metadata disagrees.
    - This data block can potentially be allocated to a second file, leading to the same outcome as case three.
6) The BITMAP and INODE are written, but not the DATA BLOCK.
    - The system will be consistent since the meta data agrees.
    - The user loses data and the file is likely corrupted since one of the data blocks in
    the file contains garbage.


# How to maintain consistency despite unexpected crashes?

One way is to fix inconsistencies on reboot. Scan all inodes and bitmaps and update the
bitmaps if the inodes don't agree.

Of course, this is an extremely expensive price to pay just to fix a single inconsistency.

The linux utility `fsck` does exactly this. On small systems, it could add seconds or minutes to book time. On large system, hours or days. 

Not ideal. A better way:

# A Journaling File System

To prevent inconsistencies, the file system maintains a journal. 

When a request comes along, first record what is about to happen, then actually do it.

After a poweroff, the system checks the journal and completes any pending requests.

If a crash happens in the middle of writing to the journal, that write is lost, but the system is consistent.

If a crash happens during writing to the disk, on reboot, the journal entry is replayed and completed.

The OS writes **transactions** to the journal. A **transaction** has a header and tail. If the tail is missing, then the system crashed in the middle of writing to the journal.

Between the header and tail, the system writes the inodes, bitmaps, and data blocks to be written to disk.

This is not efficient, it effectively halves the speed of the disk by writing everything out twice.